In [15]:
# import the necessary libraries
import numpy as np
import cv2
import dlib

In [19]:
def landmark_detection(image):
    """Generate facial landmark points of a give image.

    Parameters
    ----------
    image : str
        image file path.

    Returns
    -------
    landmark_points : list
        return a list of tuple integer of the landmark points.
    img : numpy array
        return an image where the landmark points draw on the the image.

    """

    # laod an image then convert it to grey scale
    img = cv2.imread(image)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # detect the face then find the landmarks
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

    faces = detector(img_gray)
    faces_landmark_points = []
    for face in faces:
        landmarks = predictor(img_gray, face)
        landmark_points = []
        for n in range(68):
            x = landmarks.part(n).x
            y = landmarks.part(n).y
            landmark_points.append((int(x), int(y)))

            cv2.circle(img, (x, y), 3, (0, 0, 255), -1)

        faces_landmark_points.append(landmark_points)

    return faces_landmark_points, img

In [20]:
image = "images/original_images/ted_cruz.jpg"
faces_landmark_points, landmark_points_img = landmark_detection(image)

# display the landmark points
cv2.imshow("landmark points", landmark_points_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [21]:
np.array(faces_landmark_points).shape

(1, 68, 2)

In [50]:
image1 = 'images/original_images/jim-carrey-on-the-car-oh-really-meme.jpg'
image2 = 'images/original_images/young_couple.jpg'

In [51]:
# find landmark points of the images
faces_landmark_points1, landmarks_img1 = landmark_detection(image1)
faces_landmark_points2, landmarks_img2 = landmark_detection(image2)

In [52]:
cv2.imshow("landmarks_img1", landmarks_img1)
cv2.imshow("landmarks_img2", landmarks_img2)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [53]:
print("shape of landmark points1: ", np.array(faces_landmark_points1).shape)
print("shape of landmark points2: ", np.array(faces_landmark_points2).shape)

shape of landmark points1:  (1, 68, 2)
shape of landmark points2:  (2, 68, 2)


In [ ]:
def applyConvexHull(points1, points2):
    """Find the convex hull of each landmark points.

    Parameters
    ----------
    points1 : list
        a list of tuple integer of landmark points 1.
    points2 : list
        a list of tuple integer of landmark points 2.

    Returns
    -------
    hull1 : list
        return a list of tuple integer of convex hull points bounding landmark points 1.
    hull2 : list
        return a list of tuple integer of convex hull points bounding landmark points 2.
    """

    # Find convex hull of the two landmark points
    hull1 = []
    hull2 = []

    hullIndex1 = cv2.convexHull(np.array(points1), returnPoints=False)
    hullIndex2 = cv2.convexHull(np.array(points2), returnPoints=False)

    # use the minimum number of convex hull points to avoid index out of bound.
    if len(hullIndex1) < len(hullIndex2):
        hullIndex = hullIndex1
    else:
        hullIndex = hullIndex2

    for i in range(len(hullIndex)):
        hull1.append(points1[int(hullIndex[i])])
        hull2.append(points2[int(hullIndex[i])])

    return hull1, hull2

In [54]:

def applyConvexHull(faces_landmark_points1, faces_landmark_points2):
    """Find the convex hull of each landmark points.

    Parameters
    ----------
    points1 : list
        a list of tuple integer of landmark points 1.
    points2 : list
        a list of tuple integer of landmark points 2.

    Returns
    -------
    hull1 : list
        return a list of tuple integer of convex hull points bounding landmark points 1.
    hull2 : list
        return a list of tuple integer of convex hull points bounding landmark points 2.
    """

    faces1 = np.array(faces_landmark_points1).shape[0]
    faces2 = np.array(faces_landmark_points2).shape[0]

    if faces2 > faces1:
        faces = faces2
    else:
        faces = faces1

    faces_hull1 = []
    faces_hull2 = []
    for face in range(faces):
        # Find convex hull of the two landmark points
        hull1 = []
        hull2 = []

        landmark_points1 = faces_landmark_points1[face]
        landmark_points2 = faces_landmark_points2[face]

        hullIndex1 = cv2.convexHull(
            np.array(landmark_points1), returnPoints=False)
        hullIndex2 = cv2.convexHull(
            np.array(landmark_points2), returnPoints=False)

        # use the minimum number of convex hull points to avoid index out of bound.
        if len(hullIndex1) < len(hullIndex2):
            hullIndex = hullIndex1
        else:
            hullIndex = hullIndex2

        for i in range(len(hullIndex)):
            hull1.append(landmark_points2[int(hullIndex[i])])
            hull2.append(landmark_points2[int(hullIndex[i])])

        faces_hull1.append(hull1)
        faces_hull2.append(hull2)

    return faces_hull1, faces_hull2

In [55]:
# find the convex hull bounding the landmark points of the images
# faces_hull1, faces_hull2 = applyConvexHull(faces_landmark_points1, faces_landmark_points2)
dict = {}
for i in land:
    faces_hull1, faces_hull2 = applyConvexHull(faces_landmark_points1, faces_landmark_points2)
    dict[i]=[faces_hull1, faces_hull2]

    

IndexError: list index out of range

In [49]:
print("length hull1: ", np.array(faces_hull1).shape)
print("length hull2: ", np.array(faces_hull2).shape)

length hull1:  (1, 20, 2)
length hull2:  (1, 20, 2)


In [10]:
def approachs(approach, hull1, hull2, landmark_points1, landmark_points2):
    """use convexHull or landmark_points to calculate delauney triangulation.

    Parameters
    ----------
    approach : str
        takes either 'approach1' for convexHUll or 'approach2' for landmark points.
    hull1 : list
        a list of tuple integer of convex hull points bounding landmark points 1
    hull2 : type
        a list of tuple integer of convex hull points bounding landmark points 2
    landmark_points1 : list
        a list of tuple integer of landmark points of image1
    landmark_points2 : list
        a list of tuple integer of landmark points of image2.

    Returns
    -------
    points1 : list
        a list of tuple integer of either hull 1 or landmark points 1.
    points2 : list
        a list of tuple integer of either hull 2 or landmark points 2.

    """

    # use the two approachs
    if approach == "approach1":
        points1 = hull1
        points2 = hull2
    else:
        points1 = landmark_points1
        points2 = landmark_points2

    return points1, points2

In [11]:
approach = "approach1"
points1, points2 = approachs(approach, hull1, hull2, landmark_points1, landmark_points2)

In [12]:
def calculateDelaunayTriangles(image, points):
    """Short summary.

    Parameters
    ----------
    image : type
        Description of parameter `image`.
    points : type
        Description of parameter `points`.

    Returns
    -------
    type
        Description of returned object.

    """

    img = cv2.imread(image)
    rect = (0, 0, img.shape[1], img.shape[0])

    subdiv = cv2.Subdiv2D(rect)
    subdiv.insert(points)
    triangleList = subdiv.getTriangleList()

    delaunayTri_indexes = []


    for t in triangleList:

        pt1 = (t[0], t[1])
        pt2 = (t[2], t[3])
        pt3 = (t[4], t[5])
        pts = [pt1, pt2, pt3]

        # draw the Triangulation
        cv2.line(img, pt1, pt2, (0, 0, 255), 2)
        cv2.line(img, pt2, pt3, (0, 0, 255), 2)
        cv2.line(img, pt1, pt3, (0, 0, 255), 2)

        index = []
        #Get face-points (from 68 face detector) by coordinates
        for i in range(3):
            for j in range(0, len(points)):
                if(pts[i][0] == points[j][0] and pts[i][1] == points[j][1]):
                    index.append(j)
        # Three points form a triangle
        if len(index) == 3:
            delaunayTri_indexes.append((index[0], index[1], index[2]))

    return delaunayTri_indexes, img

In [13]:
# calculate the delauney triangulations
triangulation_indexes1, triangulation_img1 = calculateDelaunayTriangles(image1, points1)
triangulation_indexes2, triangulation_img2 = calculateDelaunayTriangles(image2, points2)

In [15]:
type(triangulation_indexes1)

list

In [16]:
type(triangulation_img1)

numpy.ndarray

In [17]:
def applyAffineTransform(src, srcTri, dstTri, dsize) :

    # Given a pair of triangles, find the affine transform.
    warpMat = cv2.getAffineTransform( np.float32(srcTri), np.float32(dstTri) )

    # Apply the Affine Transform just found to the src image
    dst = cv2.warpAffine( src, warpMat, (dsize[0], dsize[1]), None, flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT_101 )

    return dst

In [19]:
help(cv2.getAffineTransform)

Help on built-in function getAffineTransform:

getAffineTransform(...)
    getAffineTransform(src, dst) -> retval
    .   @overload



In [21]:
# load and read the images
img1 = cv2.imread(image1)
img2 = cv2.imread(image2)
img2_original = np.copy(img2)

In [23]:
type(img1)

numpy.ndarray

In [24]:
cv2.imread?

In [25]:
def applyAffineTransform(src, srcTri, dstTri, dsize) :
    """Short summary.

    Parameters
    ----------
    src : type
        Description of parameter `src`.
    srcTri : type
        Description of parameter `srcTri`.
    dstTri : type
        Description of parameter `dstTri`.
    dsize : type
        Description of parameter `dsize`.

    Returns
    -------
    type
        Description of returned object.

    """

    # Given a pair of triangles, find the affine transform.
    warpMat = cv2.getAffineTransform( np.float32(srcTri), np.float32(dstTri) )

    # Apply the Affine Transform just found to the src image
    dst = cv2.warpAffine( src, warpMat, (dsize[0], dsize[1]), None, flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT_101 )

    return dst

In [26]:

def warpTriangle(img1, img2, t1, t2) :
    """Short summary.

    Parameters
    ----------
    img1 : numpy.ndarray
        output of image1 read by opencv.
    img2 : numpy.ndarray
        output of image2 read by opencv.
    t1 : tuple
        list of tuple of the triangle points.
    t2 : tuple
        list of tuple of the triangle points.

    Returns
    -------
    type
        Description of returned object.

    """

    # Find bounding rectangle for each triangle
    r1 = cv2.boundingRect(np.array(t1))
    r2 = cv2.boundingRect(np.array(t2))

    # Offset points by left top corner of the respective rectangles
    t1Rect = []
    t2Rect = []

    for i in range(3):
        t1Rect.append(((t1[i][0] - r1[0]),(t1[i][1] - r1[1])))
        t2Rect.append(((t2[i][0] - r2[0]),(t2[i][1] - r2[1])))

    # Apply warpImage to small rectangular patches
    img1Rect = img1[r1[1]:r1[1] + r1[3], r1[0]:r1[0] + r1[2]]

    size = (r2[2], r2[3])
    img2Rect = applyAffineTransform(img1Rect, t1Rect, t2Rect, size)

    # Get mask by filling triangle
    mask = np.zeros((r2[3], r2[2], 3), dtype = np.float32)
    cv2.fillConvexPoly(mask, np.int32(t2Rect), (1.0, 1.0, 1.0));
    img2Rect = img2Rect * mask

    # Copy triangular region of the rectangular patch to the output image
    img2[r2[1]:r2[1]+r2[3], r2[0]:r2[0]+r2[2]] = img2[r2[1]:r2[1]+r2[3], r2[0]:r2[0]+r2[2]] * ( (1.0, 1.0, 1.0) - mask ) + img2Rect
    # img2[r2[1]:r2[1]+r2[3], r2[0]:r2[0]+r2[2]] = img2[r2[1]:r2[1]+r2[3], r2[0]:r2[0]+r2[2]] + img2Rect


In [27]:
def applyWarpTriangle(img1, img2, img1Tri, img2Tri, points1, points2):
    """compute warp triangles for each triangles of image1 and image2.first find 
       corresponding landmark points of each triangles from the triangulations 
       indexes. then warp each triangle of image1 to image2.

    Parameters
    ----------
    img1 : numpy.ndarray
        output of image1 read by opencv.
    img2 : numpy.ndarray
        output of image2 read by opencv.
    img2Tri : list
        delaunay triangle indexes of image2.
    points1 : list
        landmark points of image1.
    points2 : list
        landmark points of image2.

    Returns
    -------
    img2 : numpy.ndarray
        warped img1 copied to img2.

    """

    # iterate through each triangles
    for i in range(len(img2Tri)):
        t1 = []
        t2 = []

        # iterate through all the three triangle indexes and find t1 and t2
        for j in range(3):
            t1.append(points1[img1Tri[i][j]])
            t2.append(points2[img2Tri[i][j]])

        warpTriangle(img1, img2, t1, t2)

    return img2

In [28]:
img2warped = applyWarpTriangle(img1, img2, triangulation_indexes1, triangulation_indexes2, points1, points2)

In [32]:
type(img2warped)

numpy.ndarray